In [1]:
from a_protocol_0.utils.decorators import log

class Test():
    
    @log
    def test_log(self, a, b, c=2, d=None):
        print("exec test_log")
        
t = Test()

t.test_log("toto", "titi", c="42")

AttributeError: 'NoneType' object has no attribute 'log_debug'

In [ ]:
import time

start_at = time.time()
time.sleep(1.2323)
end_at = time.time()

print("%.3f" % (start_at - end_at))

In [ ]:
import traceback

try:
    raise RuntimeError("hello")
except (Exception, RuntimeError):
    print(traceback.format_exc().splitlines())


In [ ]:
   # in protocol0 __init__
    # self.start = time.time()
        #
        # def _tick():
        #     self.log_info("_tick call in %.3f" % (time.time() - self.start))
        #     self.start = time.time()

        # _scheduler = Live.Base.Timer(callback=_tick, interval=1, repeat=True)
        # # _scheduler.start()
        # self.log_debug(self._tasks)
        # # self._tasks.add(
        # #     sequence(wait(0.3), run(_scheduler.restart))).kill()
        # for i in range(20):
        #     self._tasks.add(
        #         sequence(wait(i / 10), run(_tick)))
        # self.log_debug("scheduler started")

In [ ]:
"toto".splitlines()

In [ ]:
"%s terminated in %.3fs" % ("toto", 2.3232)

In [ ]:
from functools import partial
from collections import deque, Iterable


class Toto():
    pass

t = Toto()
t.titi = 2

setattr(t, "titi", 3)

f = partial(setattr, t, "titi", 4)
f()

print(t.titi)



isinstance(lambda: True, Iterable)

In [ ]:
from collections import deque, Iterable
steps = deque([1, 2, 3])
steps[-1]

print(0 or 0)

In [ ]:
from functools import partial
from a_protocol_0.sequence.Sequence import Sequence
from __future__ import print_function

from a_protocol_0 import Protocol0, EmptyModule
p0 = Protocol0(EmptyModule(is_false=False), init_song=False)

def seq_creator(i):
    if i == 0:
        return None
    return Sequence().add(partial(seq_creator, i - 1), name="seq %s" % i).done()

def check_function(res):
    print("check and return %s" % res)
    return res

def check_seq(res):
    seq = Sequence(name="check seq")
    seq.add(partial(check_function, res))
    return seq.done()

def inner_print_seq(message):
    seq = Sequence(name="inner print seq")
    seq.add(lambda: print(message))
    return seq.done()

def print_seq(message):
    seq = Sequence(name="print seq")
    seq.add(partial(inner_print_seq, message))
    return seq.done()

with p0.component_guard():
#     seq = Sequence(name="test seq")
#     seq.add(partial(seq_creator, 2))
#     seq.done()()
#     check_seq(True)()
#     print("------")
    
#     seq = Sequence(name="main").add(partial(print_seq, "titi messaged"), do_if=partial(check_seq, True))
#     seq.done()()

    seq = p0.protocol0_song.select_track(EmptyModule(), sync=True)
    print(seq._auto_start)
    
# p0.actionManager.action_scroll_tracks(go_next=True)



In [ ]:
arr = []
any(i for i in ([1] + arr) > 2)

In [ ]:
from a_protocol_0.lom.Note import Note
from a_protocol_0 import Protocol0, EmptyModule
import copy

note = Note(pitch=20, duration=2, start=0, velocity=127, muted=True, clip=EmptyModule())

note2 = copy.copy(note)
print(note)
note2._pitch = 0
print(note2)

In [ ]:
def double(L):
    for x in L:
        yield x*2
        
g = double(range(0, 5))
print(g)
print(list(g))

In [2]:
from copy import copy
from a_protocol_0.lom.Note import Note
from __future__ import print_function
from fractions import Fraction
from _Framework.Util import find_if



class Auto:
    def __init__(self, *a, **k):
        self.ramping_steps = 7
        self.ramping_duration = 0.25  # eighth note
        

    def _ramp_two_notes(self, note, next_note):
        # type: (Note, Note) -> List[Note]
        """
            2 cases : when the note is long and ramping happens at the end
            or when the note is short and the ramping takes the whole note duration
        """
        if note.velocity <= next_note.velocity:
            yield note
            return

        above_ramping_duration = note.duration > self.ramping_duration * (1 + Fraction(1, self.ramping_steps))
        print(above_ramping_duration)
        if above_ramping_duration:
            ramping_steps = self.ramping_steps - 1
            start_coeff = 1 - Fraction(1, self.ramping_steps)
            ramp_start = note.start + note.duration - self.ramping_duration * start_coeff
            velocity_start = note.velocity * start_coeff
            base_duration = self.ramping_duration * start_coeff

            first_note = copy(note)
            first_note.duration = note.duration - self.ramping_duration + self.ramping_duration / self.ramping_steps
            yield first_note
        else:
            ramping_steps = self.ramping_steps
            ramp_start = note.start
            velocity_start = note.velocity
            base_duration = note.duration

        for i in range(0, ramping_steps):
            coeff = float(i) / ramping_steps
            ramp_note = copy(note)
            ramp_note.start = ramp_start + (base_duration * coeff)
            ramp_note.duration = base_duration / ramping_steps
            ramp_note.velocity = round(velocity_start - (velocity_start - next_note.velocity) * coeff)
            yield ramp_note
            
        print(ramping_steps, ramp_start, base_duration)
        
    def _consolidate_notes(self, notes, clip_length):
        # type: (List[Note], float) -> List[Note]
        # notes_by_start = {}
        # for note in notes:
        #     if not notes_by_start.has_key(note.start):
        #         notes_by_start[note.start] = note
        #     else:
        #         notes_by_start[note.start] = max([notes_by_start[note.start], note], key=lambda n: n.duration)
        #
        # single_notes = notes_by_start.values()

        for i, next_note in enumerate(notes[1:] + [Note(start=clip_length)]):
            current_note = notes[i]
            current_note.duration = next_note.start - current_note.start

        current_note = notes[0]
        yield current_note
        for next_note in notes[1:]:
            if next_note.velocity == current_note.velocity:
                current_note.duration += next_note.duration
            else:
                current_note = next_note
                yield current_note
                
    def _insert_added_note(self, notes, added_note):
        # type: (List[Note], Note) -> List[Note]
        # we could use clip.start here
        if notes[0].start != 0:
            notes[0].duration += notes[0].start
            notes[0].start = 0

        i = 0
        while i < len(notes):
            note = notes[i]
            if not note.includes_start(added_note):
                yield note
                i += 1
                continue

            if note.start != added_note.start:
                truncated_note = copy(note)
                truncated_note.end = added_note.start
                yield truncated_note

            while added_note.end >= note.end and i < len(notes) - 1:
                i += 1
                note = notes[i]
            yield added_note
            print(added_note.end, note.end, note.duration)
            if added_note.end < note.end:
                note.duration -= (added_note.end - note.start)
                note.start = added_note.end
                yield note
                
            i += 1
            
    def _remove_ramped_notes_start(self, notes):
        # type: (List[Note]) -> List[Note]
        """ clean ramped clip (remove un_quantized and duplicated on start notes) """
        notes_by_start = {}
        for note in notes:
            if not notes_by_start.has_key(note.start):
                notes_by_start[note.start] = note
            else:
                notes_by_start[note.start] = max([notes_by_start[note.start], note], key=lambda n: n.duration)

        return notes_by_start.values()

            
auto = Auto()
Note.auto_sync_enabled = False


# note_1 = Note(start=1.8, duration=0.2, pitch=100, velocity=100)
# note_2 = Note(start=0, duration=1, pitch=0, velocity=0)
# note_3 = Note(start=1, duration=1, pitch=0, velocity=0)
# note_4 = Note(start=2, duration=1, pitch=100, velocity=100)
note_4 = Note(start=0, duration=0.5, pitch=100, velocity=100)
note_4_b = Note(start=0, duration=0.01, pitch=100, velocity=100)
note_5 = Note(start=0.75, duration=0.5, pitch=80, velocity=80)
added_note = Note(start=0, duration=1, pitch=20, velocity=20)

print(list(auto._remove_ramped_notes_start([note_4, note_4_b])))


[{start:0, duration:0.5, pitch:100, vel:100}]


In [4]:
from _Framework.Util import find_if

rack_name = "toto"
res = find_if(lambda d: d.name == rack_name, [])

print(res)

None
